In [7]:
#model using all areas
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
#from google.colab import drive

# Mount Google Drive
#drive.mount('/cdata = pd.read_csv("covtype.csv")

# Load the data
#data = pd.read_csv('/content/drive/MyDrive/covtype.csv')
data = pd.read_csv("covtype.csv")

# Binarize the target:
# Set target = 1 if Horizontal_Distance_To_Fire_Points < 1600, else 0.
data['Horizontal_Distance_To_Fire_Points'] = np.where(
    data['Horizontal_Distance_To_Fire_Points'] < 1600, 1, 0
)

# Drop Cover_Type since it's a different target and not needed for our fire model
data.drop(columns=['Cover_Type'], inplace=True)

# Prepare features and target using all areas
X = data.drop(columns=['Horizontal_Distance_To_Fire_Points'])
Y = data['Horizontal_Distance_To_Fire_Points']

# Scale the features to [0,1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets (using stratification for balanced classes)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.25, random_state=42, stratify=Y
)

# Define a parameter grid that respects solver/penalty compatibilities:
param_grid = [
    # For solvers that support only 'l2' or no penalty:
    {
        'solver': ['lbfgs', 'newton-cg', 'sag'],
        'penalty': ['l2', None],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    },
    # For liblinear, which supports 'l1' and 'l2':
    {
        'solver': ['liblinear'],
        'penalty': ['l1', 'l2'],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    },
    # For saga, which supports 'l1', 'l2', 'elasticnet', or no penalty:
    {
        'solver': ['saga'],
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    }
]

# Initialize GridSearchCV with 3-fold cross-validation
grid_search = GridSearchCV(
    LogisticRegression(),
    param_grid=param_grid,
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Run grid search on the training data
best_clf = grid_search.fit(X_train, Y_train)

# Display the best estimator and its training accuracy
print("Best Estimator:")
print(best_clf.best_estimator_)
print(f"Training Accuracy: {best_clf.score(X_train, Y_train):.3f}")

# Evaluate performance on the test set
Y_pred = best_clf.predict(X_test)
print("Test Set Performance:")
print(classification_report(Y_test, Y_pred))


Fitting 3 folds for each of 168 candidates, totalling 504 fits
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   1.0s
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   1.3s
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   1.3s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   1.6s
[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   1.6s
[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   1.7s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END ......C=0.001, max_iter=100, penalty=l2, solver=sag; total time=   5.1s
[CV] END ..C=0.001, max_iter=100, penalty=None, solver=lbfgs; total time=   3.8s
[CV] END ..C=0.001, max_iter=100, penalty=None, solver=lbfgs; total time=   3.9s


...

[CV] END ..C=100.0, max_iter=1000, penalty=None, solver=saga; total time= 4.3min


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END ...C=1000.0, max_iter=1000, penalty=l2, solver=saga; total time= 3.1min
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time= 4.4min
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time= 4.4min
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time= 4.4min
[CV] END ...C=1000.0, max_iter=1000, penalty=l2, solver=saga; total time= 2.6min
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time= 3.3min
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time= 3.3min
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time= 3.3min


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
42 fits failed out of a total of 504.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
42 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py", line 1204, in fit
    raise

Best Estimator:
LogisticRegression(C=0.01, max_iter=1000, penalty='l1', solver='saga')
Training Accuracy: 0.681
Test Set Performance:
              precision    recall  f1-score   support

           0       0.68      0.77      0.72     78062
           1       0.68      0.58      0.63     67191

    accuracy                           0.68    145253
   macro avg       0.68      0.68      0.68    145253
weighted avg       0.68      0.68      0.68    145253



In [9]:
#model using areas 2 and 4
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Load the data
data = pd.read_csv('covtype.csv')

data['Horizontal_Distance_To_Fire_Points'] = np.where(
    data['Horizontal_Distance_To_Fire_Points'] < 1600, 1, 0
)

data.drop(columns=['Cover_Type'], inplace=True)

subsetData_Area24 = data[~(data['Wilderness_Area1'] == 1) & ~(data['Wilderness_Area3'] == 1)]

X24 = subsetData_Area24.drop(columns=['Horizontal_Distance_To_Fire_Points'])
Y24 = subsetData_Area24['Horizontal_Distance_To_Fire_Points']

scaler2 = MinMaxScaler()
X24 = scaler2.fit_transform(X24)

X24_train, X24_test, Y24_train, Y24_test = train_test_split(
    X24, Y24, test_size=0.25, random_state=42, stratify=Y24
)

# seperated to avoid errors
param_grid_24 = [
    {
        'solver': ['lbfgs', 'newton-cg', 'sag'],
        'penalty': ['l2', None],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    },
    {
        'solver': ['liblinear'],
        'penalty': ['l1', 'l2'],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    },
    {
        'solver': ['saga'],
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'C': np.logspace(-3, 3, 7),
        'max_iter': [100, 1000]
    }
]

grid_search_24 = GridSearchCV(
    LogisticRegression(),
    param_grid=param_grid_24,
    cv=3,
    verbose=2,
    n_jobs=-1
)

best_clf_24 = grid_search_24.fit(X24_train, Y24_train)

print("Best Estimator for Areas 2 and 4:")
print(best_clf_24.best_estimator_)
print(f"Training Accuracy: {best_clf_24.score(X24_train, Y24_train):.3f}")


Y24_pred = best_clf_24.predict(X24_test)
print("Test Set Performance for Areas 2 and 4:")
print(classification_report(Y24_test, Y24_pred))


Fitting 3 folds for each of 168 candidates, totalling 504 fits
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END ....C=0.001, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.2s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END ......C=0.001, max_iter=100, penalty=l2, solver=sag; total time=   0.8s
[CV] END ......C=0.001, max_iter=100, penalty=l2, solver=sag; total time=   0.9s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (st

[CV] END ..C=0.001, max_iter=100, penalty=None, solver=lbfgs; total time=   0.9s
[CV] END ..C=0.001, max_iter=100, penalty=None, solver=lbfgs; total time=   0.9s
[CV] END ..C=0.001, max_iter=100, penalty=None, solver=lbfgs; total time=   0.9s
[CV] END ......C=0.001, max_iter=100, penalty=l2, solver=sag; total time=   1.1s
[CV] END C=0.001, max_iter=100, penalty=None, solver=newton-cg; total time=   1.0s


...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END ...C=1000.0, max_iter=1000, penalty=l2, solver=saga; total time=  19.1s
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time=  27.5s
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time=  25.5s
[CV] END ...C=1000.0, max_iter=1000, penalty=l1, solver=saga; total time=  28.8s
[CV] END ...C=1000.0, max_iter=1000, penalty=l2, solver=saga; total time=  17.0s
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time=  20.1s
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time=  19.7s
[CV] END .C=1000.0, max_iter=1000, penalty=None, solver=saga; total time=  15.4s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
42 fits failed out of a total of 504.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
42 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py", line 1204, in fit
    raise

Best Estimator for Areas 2 and 4:
LogisticRegression(C=0.001, penalty=None)
Training Accuracy: 0.875
Test Set Performance for Areas 2 and 4:
              precision    recall  f1-score   support

           0       0.78      0.75      0.76      4440
           1       0.91      0.92      0.92     12273

    accuracy                           0.88     16713
   macro avg       0.84      0.84      0.84     16713
weighted avg       0.88      0.88      0.88     16713



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
